In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import yaml
import math
import h5py
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from eventsFactory import getEvents
from reco import getRecoResults

In [8]:
RUN         = 1231

DATA_PATH   = "../data/"
DATA_FILE   = DATA_PATH + f"RUN00{RUN}_data.txt"

CONFIG_PATH = "../config/"
CONFIG_FILE = CONFIG_PATH + f"RUN00{RUN}.yml"

In [6]:
stream_df = pd.read_csv(DATA_FILE)
stream_df.head(10)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,2,1,129,52697671.0,1273.0,6.0
1,2,1,129,52697681.0,1290.0,22.0
2,2,1,129,52697685.0,1530.0,21.0
3,2,1,129,52697686.0,1959.0,7.0
4,2,1,129,52697693.0,2154.0,12.0
5,2,1,129,52697695.0,17.0,1.0
6,2,1,129,52697695.0,892.0,22.0
7,2,1,129,52697696.0,646.0,11.0
8,2,0,11,52697698.0,1586.0,27.0
9,2,0,11,52697698.0,1598.0,25.0


In [9]:
with open(CONFIG_FILE, "r") as config_file:
    cfg = yaml.safe_load(config_file)

In [10]:
useTrigger = False # what does this parameter do??
runTimeShift = 0   # what does this parameter do??

keep = ["FPGA", "TDC_CHANNEL", "HIT_DRIFT_TIME", "m"]

In [12]:
df = pd.DataFrame()
# reco
events = getEvents(stream_df, cfg, runTimeShift, useTrigger)
resultsList, resultsData, resultsHits, resultsDf = getRecoResults(events, useTrigger)
# out df
for df_ in resultsDf:   
    df_ = df_[keep]
    df = pd.concat([df, df_], axis=0, ignore_index=True)

MemoryError: Unable to allocate 1.88 GiB for an array with shape (3, 84056535) and data type float64

In [ ]:
df.loc[(df["FPGA"]==0), "CH"] = df["TDC_CHANNEL"]
df.loc[(df["FPGA"]==1), "CH"] = df["TDC_CHANNEL"]+128
df = df.drop(["FPGA", "TDC_CHANNEL"], axis=1)
df["CH"]=df["CH"].astype(np.uint32)

df = df[["CH", "HIT_DRIFT_TIME", "m"]]
df = df[(df["HIT_DRIFT_TIME"]>-200) & (df["HIT_DRIFT_TIME"]<600)]
df["THETA"] = np.arctan(df["m"])*180./math.pi

df.head(10)

In [ ]:
wires = []

for wire in np.unique(df["CH"]):
    wires.append(df[df["CH"]==wire])
    df[df["CH"]==wire].to_hdf('test_wires.h5', key=f'wire{wire}', mode='a')

In [ ]:
f = h5py.File('test_wires.h5', 'r')
f.keys()

In [ ]:
full_df = pd.concat(
    [
        pd.read_hdf('test_wires.h5', key=k, mode="r") for k in f.keys()
    ],
    ignore_index=True
)

In [ ]:
for key in f.keys():
    wire_df = pd.read_hdf('test_wires.h5', key=key, mode="r")
    plt.hist(
        wire_df["HIT_DRIFT_TIME"],
        bins=100,
        range=(-190, 490)
    )
    plt.title(f"wire {wire_df.iloc[0, 0]}")
    plt.xlabel("drift time (ns)")
    plt.ylabel("counts")
    plt.show()